In [1]:
import requests
import pandas as pd

def get_series(series_id, start, end, api_key='fbf2a3cac76ec733ee2b8c01ab036950', file_type='json'):
    url_base = 'https://api.stlouisfed.org/fred/series/observations'
    url = f'{url_base}?series_id={series_id}'
    if start is not None:
        start = pd.to_datetime(start, errors='raise')
        url += '&observation_start=' + start.strftime('%Y-%m-%d')
    if end is not None:
        end = pd.to_datetime(end, errors='raise')
        url += '&observation_end=' + end.strftime('%Y-%m-%d')
    
    url += f'&api_key={api_key}&file_type={file_type}'
    
    try:
        resp = requests.get(url)
    except  Exception as e:
        print(e)
        return None
    
    dfdict = {}
    for i in resp.json()['observations']:
        dt = pd.to_datetime(i['date'], errors='raise')
        if i['value'] != '.':
            rate = float(i['value'])
        else:
            rate = None
        dfdict[dt] = rate
        
    df = pd.DataFrame.from_dict(dfdict, orient='index', columns=[series_id])
    df.fillna(method='ffill', inplace=True)
    return df

df = get_series('DEXUSUK', '1983-01-01', '2022-12-31')
df.head()

,DEXUSUK
1983-01-03,1.6235
1983-01-04,1.6210
1983-01-05,1.6210
1983-01-06,1.6065
1983-01-07,1.6100


In [2]:
df.isna().value_counts()

DEXUSUK
False      10435
dtype: int64

In [3]:
import plotly.graph_objects as go

trace = go.Scatter(x=df.index, y=df['DEXUSUK'], mode='lines')
data = [trace]
layout = {
    'title': 'USD/GBP Exchange Rate',
    'xaxis_title': 'Date',
    'yaxis_title': 'USD/GBP Rate',
    'width': 2400,
    'height': 1200,
    'hovermode': 'x unified',
    'template': 'seaborn',
    'xaxis': {
        'rangeslider': {
            'visible': True,
        },
    }
}

fig = go.Figure(data=data, layout=layout)
fig.show()

In [4]:
df.resample('W').mean()

,DEXUSUK
1983-01-09,1.61640
1983-01-16,1.58150
1983-01-23,1.57570
1983-01-30,1.54000
1983-02-06,1.52124
...,...
2022-12-04,1.21086
2022-12-11,1.22222
2022-12-18,1.22884
2022-12-25,1.20968


In [5]:
# Calculate seasonal decomposition on the DEXUSUK time series
from statsmodels.tsa.seasonal import seasonal_decompose

# decomp = seasonal_decompose(df)
decomp = seasonal_decompose(df.resample('M').mean(), extrapolate_trend='freq')

# Show decomposition plots
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residuals'])

fig.add_trace(go.Scatter(x=decomp.observed.index, y=decomp.observed.values, name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=decomp.trend.index, y=decomp.trend.values, name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=decomp.seasonal.index, y=decomp.seasonal.values, name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=decomp.resid.index, y=decomp.resid.values, name='Residuals'), row=4, col=1)

fig.update_layout(width=2200, height=1400, title='Seasonal Decomposition Plot', template='seaborn')

# fig['layout']['yaxis3'].update(range=[0,2])

fig.show()

In [21]:
from pmdarima import auto_arima

arimafit = auto_arima(df['DEXUSUK'].dropna(), trace=True)
arimafit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-67558.703, Time=4.55 sec


In [17]:
# train = df['DEXUSUK'].iloc[:-100]
# test = df['DEXUSUK'].iloc[-100:]
train = df['DEXUSUK'].resample('W').mean().iloc[:-100]
test = df['DEXUSUK'].resample('W').mean().iloc[-100:]

In [19]:
from statsmodels.tsa.arima.model import ARIMA

# mod = ARIMA(train, order=(1,1,0), freq='C')
mod = ARIMA(train, order=(0,1,1))
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                DEXUSUK   No. Observations:                 1987
Model:                 ARIMA(0, 1, 1)   Log Likelihood                5173.844
Date:                Sun, 06 Aug 2023   AIC                         -10343.688
Time:                        14:39:48   BIC                         -10332.500
Sample:                    01-09-1983   HQIC                        -10339.579
                         - 01-31-2021                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.2530      0.015     16.366      0.000       0.223       0.283
sigma2         0.0003   5.25e-06     60.883      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              2652.93
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.63   Skew:                            -0.78
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.44
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [20]:
res.predict(start=len(train), end=len(train)+len(test)).value_counts()

1.371939    101
Name: predicted_mean, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_predict

def arima_forecast_plot(model, train, test, title, ylabel, xlabel, start=len(train), end=len(train)+len(test), figsize=(16, 8), plt_ext=-200):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    train.plot(ax=ax)
    test.plot(ax=ax, color='teal')
    plot_predict(model, start=start, end=end, ax=ax, alpha=0.05)
    
    ax.legend(['Training Data', 'Testing Data', 'Predictions'])
    plt_range_start = train.index[plt_ext]
    plt_range_end = test.index[-1]
    plt_max = max(train.iloc[plt_ext:].max(), test.max()) * 1.1
    plt_min = min(train.iloc[plt_ext:].min(), test.min()) * 0.9
    plt.xlim([plt_range_start, plt_range_end])
    plt.ylim([plt_min, plt_max])
    plt.show()
arima_forecast_plot(res, train, test, title='GBP/USD FX Rate Forecasts (100 Days)', ylabel='FX Rate', xlabel='Date')